In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

# pull feature engineering notebook
# add your own feature engineering functions to features.py and import them
# only rule is the function must take a dataframe and return a dataframe (with your new features)
from features import engineer, scrub, no_photo, num_keyword

color = sns.color_palette()
%matplotlib inline

df_train = pd.read_json("train.json")
df_test = pd.read_json("test.json")

### PRE-PROCESSING

In [ ]:
#engineer features (from script)
scrub_and_engineer = [scrub, engineer, no_photo, num_keyword]
for func in scrub_and_engineer:
    df_train = func(df_train)
    df_test = func(df_test)

In [ ]:
df_train.columns

### Baseline training

In [ ]:
feats_to_train = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day"]
X = df_train[feats_to_train]
y = df_train["interest_level"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20)

clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
baseline_logloss = log_loss(y_val, y_val_pred)

In [ ]:
baseline_logloss

### Training the new model

In [ ]:
feats_to_train = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day", 'n_num_keyfeat_score']
X = df_train[feats_to_train]
y = df_train["interest_level"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_val_pred = clf.predict_proba(X_val)
test_logloss = log_loss(y_val, y_val_pred)

### Testing for raw improvement

In [ ]:
test

In [ ]:
baseline_logloss

In [ ]:
baseline_logloss - test_logloss

### Clean up and output a submission file

In [ ]:
X = df_test[feats_to_train]
y = clf.predict_proba(X)

In [ ]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}

In [ ]:
sub = pd.DataFrame()
sub["listing_id"] = df_test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]

In [ ]:
sub.to_csv("submission_rf.csv", index=False)

In [ ]:
submission = pd.read_csv('submission_rf.csv')

In [ ]:
len(submission)